In [17]:
import mediapipe as mp
import cv2
from datetime import datetime
import math

In [18]:
chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
nose_tip = [1]
right_eye = [33]
left_eye = [263]
lips_upper = [61, 291]
chin = [199]
chosen_mouth_idxs = [38, 76, 268, 292, 86, 316 ]

In [19]:
facemesh =  mp.solutions.face_mesh
face = facemesh.FaceMesh(
    static_image_mode=False, 
    min_tracking_confidence=0.5, 
    min_detection_confidence=0.5
    )
draw = mp.solutions.drawing_utils

In [20]:
def eucledianDistance(point1, point2):
    return math.sqrt((point2.x - point1.x)**2 + (point2.y - point1.y)**2)

def EARandMAR(right_eye, left_eye, mouth):
    right_eye = (eucledianDistance(right_eye[2], right_eye[5]) + eucledianDistance(right_eye[4], right_eye[3]))/(2*eucledianDistance(right_eye[0], right_eye[1]))
    left_eye = (eucledianDistance(left_eye[4], left_eye[3]) + eucledianDistance(left_eye[5], left_eye[2]))/(2*eucledianDistance(left_eye[1], left_eye[0]))
    mouth = (eucledianDistance(mouth[0], mouth[2]) + eucledianDistance(mouth[3], mouth[5]))/(2*eucledianDistance(mouth[1], mouth[4]))
    return right_eye, left_eye, mouth


def diffTime(mar_timer):
    diff = (datetime.now() - mar_timer).total_seconds()
    return diff 

In [21]:
thresh_mar = 0.5
thresh_ear = 0.3

thresh_yawn_time = 7
thresh_close_eye = 2

mar_timer = None
mar_timer = None


cap =  cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgH, imgW, _ = frame.shape
    rgb =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = face.process(rgb)
    
    if op.multi_face_landmarks:
        for pt in op.multi_face_landmarks:
            # draw.draw_landmarks(frame, i)
            temp = [[],[],[]]
            left = 1
            right = 1
            mouth = 1
            head = 1
            for no,point in enumerate(pt.landmark):
                if no in chosen_left_eye_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (0, color, 0), -1)
                    cv2.putText(
                        frame, text= str(f'p{left}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[0].append(point)
                    left += 1
                if no in chosen_right_eye_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (color, 0, 0), -1)
                    cv2.putText(
                        frame, text= str(f'p{right}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[1].append(point)
                    right += 1
                if no in chosen_mouth_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (0, 0, color), -1)
                    cv2.putText(
                        frame, text= str(f'p{mouth}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[2].append(point)
                    mouth += 1
                # if no in nose_tip or no in right_eye or no in left_eye or no in lips_upper or no in chin:
                #     cv2.putText(
                #         frame, text= str(f'p{head}'), org=(int(point.x * imgW), int(point.y * imgH)),
                #         fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                #         thickness=1, lineType=cv2.LINE_AA
                #     )
                #     head += 1



            right_eye, left_eye, mar = EARandMAR(temp[0], temp[1], temp[2])
            ear = (right_eye + left_eye) / 2
            
            cv2.putText(frame, str(f'mar: {mar:.3f}'), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, str(f'ear: {ear:.3f}'), (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


            if((mar > thresh_mar) and (mar_timer == None)):
                mar_timer = datetime.now()
            elif(mar <= thresh_mar):
                mar_timer = None
            
            if mar_timer is not None :
                diff_time = diffTime(mar_timer)
                cv2.putText(frame, str(f'mar_timer: {diff_time:.3f} s'), (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            
                if diff_time > thresh_yawn_time:
                    cv2.putText(frame, str(f'ngantuk karena menguap > {thresh_yawn_time}s'), (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    # print('Pengemudi terdeteksi mengantuk karena menguap')
                # print(mar_timer, '\t', diff_time)
            
            
            if((ear < thresh_ear) and (ear_timer == None)):
                ear_timer = datetime.now()
            elif(ear >= thresh_ear):
                ear_timer = None
            
            if ear_timer is not None :
                diff_time = diffTime(ear_timer)
                cv2.putText(frame, str(f'ear_timer: {diff_time:.3f} s'), (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
                if diff_time > thresh_close_eye:
                    cv2.putText(frame, str(f'ngantuk karena mata merem > {thresh_close_eye}s'), (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            
            # print(temp[0], '\t', temp[1], '\t', temp[2])

            
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("windows", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    # break

cap.release()
cv2.destroyAllWindows()